In [1]:
import pandas as pd
import numpy as np
import cv2
import os

In [2]:
home_dir = os.getenv("HOME")
raw_data = pd.read_csv(home_dir + '/Desktop/DataPacket/data.csv')

In [3]:
raw_data.tail()

,laser0,laser1,laser2,laser3,laser4,laser5,laser6,laser7,laser8,laser9,...,laser172,laser173,laser174,laser175,laser176,laser177,laser178,laser179,speed,steering_angle
1544,4.816050,4.808475,4.825094,4.819104,4.814940,4.821913,4.838631,4.843539,4.870230,4.888047,...,1.497087,1.492515,1.470439,1.484231,1.482382,1.471577,1.472082,1.477870,1.5,-0.046
1545,4.840387,4.833663,4.838636,4.834896,4.844656,4.839835,4.864824,4.882144,4.882463,4.923291,...,1.458358,1.477810,1.470574,1.468011,1.470980,1.464960,1.466421,1.502800,1.5,-0.046
1546,4.849373,4.855845,4.851895,4.864667,4.876279,4.877153,4.899023,4.906648,4.911956,4.924626,...,1.483265,1.450763,1.456082,1.456520,1.464542,1.453171,1.446331,1.446458,1.5,-0.046
1547,4.849373,4.855845,4.851895,4.864667,4.876279,4.877153,4.899023,4.906648,4.911956,4.924626,...,1.483265,1.450763,1.456082,1.456520,1.464542,1.453171,1.446331,1.446458,1.5,-0.046
1548,4.849373,4.855845,4.851895,4.864667,4.876279,4.877153,4.899023,4.906648,4.911956,4.924626,...,1.483265,1.450763,1.456082,1.456520,1.464542,1.453171,1.446331,1.446458,1.5,-0.046


In [4]:
img_input = []
output = raw_data["steering_angle"].values

'''
from sklearn.preprocessing import scale
output = scale( output, axis=0, with_mean=True, with_std=True, copy=True )
'''

output = np.asarray(output).astype(np.float32) * 10.0
output

array([ 0.  ,  0.  ,  0.  , ..., -0.46, -0.46, -0.46], dtype=float32)

In [5]:
h = 240
w = 320
c = 1
img_num = len(raw_data) + 1  # Because csv lenght start with 0

for i in range(1, img_num):
    img = cv2.imread(home_dir + "/Desktop/DataPacket/Images/"+str(i)+".jpg", 0)
    img = cv2.resize(img, (w, h))
    
    img_input.append(img)

In [6]:
img_input = np.asarray(img_input).astype(np.float32) / 255.0

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(img_input, output, test_size=0.2, random_state=22)

del img_input
del output

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
from keras.layers import Conv2D, MaxPooling2D, Cropping2D, Lambda
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers.core import Activation

Using TensorFlow backend.


In [9]:
X_train

array([[[0.7019608 , 0.7058824 , 0.69803923, ..., 0.69803923,
         0.69803923, 0.69803923],
        [0.7019608 , 0.69411767, 0.69803923, ..., 0.7019608 ,
         0.69803923, 0.69803923],
        [0.7058824 , 0.69411767, 0.7019608 , ..., 0.7019608 ,
         0.69803923, 0.69803923],
        ...,
        [0.12941177, 0.12156863, 0.12941177, ..., 0.12941177,
         0.10980392, 0.15686275],
        [0.13725491, 0.09411765, 0.09019608, ..., 0.10588235,
         0.12941177, 0.15294118],
        [0.13333334, 0.14117648, 0.12156863, ..., 0.1254902 ,
         0.13333334, 0.14509805]],

       [[0.69803923, 0.69411767, 0.69803923, ..., 0.69803923,
         0.69803923, 0.69411767],
        [0.69411767, 0.7019608 , 0.7019608 , ..., 0.7019608 ,
         0.69803923, 0.69803923],
        [0.69411767, 0.7019608 , 0.69803923, ..., 0.69803923,
         0.69411767, 0.69803923],
        ...,
        [0.11372549, 0.13725491, 0.12941177, ..., 0.08235294,
         0.08627451, 0.1254902 ],
        [0.1

In [14]:
if len(X_train.shape) < 4:
    X_train.resize(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
    X_validation.resize(X_validation.shape[0], X_validation.shape[1], X_validation.shape[2], 1)
X_train.shape

(1239, 240, 320, 1)

In [15]:
from keras.preprocessing.image import ImageDataGenerator

dataGenerator = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=5,
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=False,
        vertical_flip=False)
dataGenerator.fit(X_train)

In [16]:
model = Sequential()

model.add(Conv2D(filters = 2, kernel_size = (3,3),padding = 'same',activation ='relu', input_shape = (h, w, c)))
model.add(Conv2D(filters = 4, kernel_size = (3,3),padding = 'same',activation ='relu'))
model.add(Conv2D(filters = 8, kernel_size = (5,5),padding = 'same',activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'same',activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 24, kernel_size = (7,7),padding = 'same',activation ='relu'))

model.add(Flatten())

'''model.add(Dense(50, activation='relu'))
model.add(Dense(28, activation='relu'))
model.add(Dense(10, activation='relu'))'''

model.add(Dense(1, activation = "sigmoid"))

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])


In [17]:
hs = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=64, epochs=3)

#hs = model.fit_generator(dataGenerator.flow(X_train, y_train, batch_size=64), epochs=3, validation_data=[X_validation, y_validation], steps_per_epoch=X_train.shape[0] // 64)

Train on 1239 samples, validate on 310 samples
Epoch 1/3
1216/1239 [============================>.] - ETA: 3s - loss: 2.2443 - acc: 0.1266 

KeyboardInterrupt: 

In [ ]:
import plotly
from plotly.offline import iplot
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go

def trainingResultsGraph(model, modelCode):
    trace0 = go.Scatter(
        x = model.epoch,
        y = model.history['loss'],
        mode = 'lines',
        name = 'loss',
        line=dict(color='aquamarine')
    )

    trace1 = go.Scatter(
        x = model.epoch,
        y = model.history['val_loss'],
        mode = 'lines',
        name = 'val_loss',
        line=dict(color='darkred', dash='dash')
    )

    trace2 = go.Scatter(
        x = model.epoch,
        y = model.history['acc'],
        mode = 'lines',
        name = 'acc',
        line=dict(color='violet')
    )

    trace3 = go.Scatter(
        x = model.epoch,
        y = model.history['val_acc'],
        mode = 'lines',
        name = 'val_acc',
        line=dict(color='aqua', dash='dash')
    )

    updatemenus = list([
        dict(type="buttons",
             active=-1,
             buttons=list([
                dict(label = 'Acc Graph',
                     method = 'update',
                     args = [{'visible': [False, False, True, True]},
                             {'title': 'Trained Model'+modelCode+' training and validation accuracy'}]),
                dict(label = 'Loss Graph',
                     method = 'update',
                     args = [{'visible': [True, True, False, False]},
                             {'title': 'Trained Model'+modelCode+' training and validation loss'}]),
                dict(label = 'Both',
                     method = 'update',
                     args = [{'visible': [True, True, True, True]},
                             {'title': 'Trained Model'+modelCode+' training and validation values'}])
            ]),
        )
    ])

    data = [trace0, trace1, trace2, trace3]
    layout = dict(title='Trained Model'+modelCode+' training and validation values',
                  xaxis = dict(title = 'Epochs'),
                  updatemenus=updatemenus)

    fig = dict(data=data, layout=layout)

    iplot(fig, filename='lossGraph')
    
trainingResultsGraph(hs, "1")

In [ ]:
import rospkg
import rospy

rospack = rospkg.RosPack()
package_path = rospack.get_path('angelshark_deeplearning')

In [ ]:
import json
import matplotlib.pylab as plt
import random
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

# Save model weights and json.
model.save_weights(package_path + '/model/model.h5')
model_json = model.to_json()
with open(package_path + '/model/model.json', 'w') as outfile:
    json.dump(model_json, outfile)

# Train and validation loss chart
print(hs.history.keys())

plt.plot(hs.history['loss'])
plt.plot(hs.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()
